# Parallel Magnetic Resonance imaging (MRI)

Standard MRI is described by the Fourier transform $\mathcal{F}$ as forward operator (here in two dimensions). 
To accelerate data acquisition, parallel MRI uses simultaneous measurements by $N$ receiver coils. This allows undersampling 
of the Fourier domain leading to speed-ups. Parallel MRI is described by the forward operator
$$F\left(\begin{array}{c}\rho \\ c_1\\ \vdots \\ c_N\end{array}\right) 
= \left(\begin{array}{c}M\cdot\mathcal{F}(c_1 \cdot \rho)\\ \vdots \\
M\cdot\mathcal{F}(c_N \cdot \rho)\end{array}\right).$$
Here $\rho$ describes the hydrogen density and is the main quantity of interest. To take into account effects such as motion artifacts, $\rho$ has to be modeled as a complex-valued function. $c_1,\dots, c_N$ describe complex-valued coil profile, which may be assumed to be smooth. As they depend on the sample $\rho$, they must be reconstructed together with $\rho$. $M$ is a 0-1-mask describing the undersampling pattern.  

In [ ]:
import logging

import matplotlib.pyplot as plt
import matplotlib as mplib
from matplotlib.colors import hsv_to_rgb
import numpy as np
from scipy.io import loadmat

import regpy.stoprules as rules
import regpy.util as uti
from examples.mri.mri import cartesian_sampling, normalize, parallel_mri, sobolev_smoother, estimate_sampling_pattern
from regpy.operators import PtwMultiplication
from regpy.solvers import RegularizationSetting
from regpy.solvers.nonlinear.irgnm import IrgnmCG
from regpy.vecsps import UniformGridFcts
from regpy.hilbert import L2
import os

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-40s :: %(message)s'
)

### Complex to rgb conversion

Converts array of complex numbers into array of RGB color values for plotting. The hue corresponds to the argument.
The brighntess corresponds to the absolute value.  

`Parameters`
>z : `numpy.ndarray`
>array of complex numbers

`Returns`
> `numpy.ndarray`
> Array that contains three values for each value in z containing the RGB representation of this value.

In [ ]:
def complex_to_rgb(z):
    HSV = np.dstack( (np.mod(np.angle(z)/(2.*np.pi),1), 1.0*np.ones(z.shape), np.abs(z)/np.max((np.abs(z[:]))), ))
    return hsv_to_rgb(HSV)

### Load data from file and estimate sampling pattern

In [ ]:
data = loadmat('data/ksp3x2.mat')['Y']
data = np.transpose(data,(2,0,1))*(100/np.linalg.norm(data))
# normalize and transpose data 
nrcoils,n1,n2 = data.shape
grid = UniformGridFcts((-1, 1, n1), (-1, 1, n2), dtype=complex)
mask = estimate_sampling_pattern(data)
plt.imshow(mask.T); plt.title('Undersampling pattern of data')

### Set up forward operator

In [ ]:
sobolev_index = 32

full_mri_op = parallel_mri(grid=grid, ncoils=nrcoils,centered=True)
sampling = PtwMultiplication(full_mri_op.codomain,(1.+0j)* mask)
#cartesian_sampling(full_mri_op.codomain, mask=mask)
smoother = sobolev_smoother(full_mri_op.domain, sobolev_index, factor=220.)

parallel_mri_op = sampling * full_mri_op * smoother

### Set up initial guess
We use constant density and zero coil profiles as initial guess.

In [ ]:
init = parallel_mri_op.domain.zeros()
init_density, _ = parallel_mri_op.domain.split(init)
init_density[...] = 1

### Set up regularization method

In [ ]:
setting = RegularizationSetting(op=parallel_mri_op, penalty=L2, data_fid=L2)

solver = IrgnmCG(
    setting=setting,
    data=data,
    regpar=1,
    regpar_step=1/3.,
    init=init
)

stoprule = rules.CountIterations(max_iterations=5) 

### Run solver by hand and plot iterates
Get an iterator from the solver

In [ ]:
it = iter(solver)

Iterate this cell by hand about 5 times

In [ ]:
"""
reco, reco_data = next(it)

rho, coils = smoother.codomain.split(smoother(reco))
#rho, coils = normalize(rho,coils)

fig = plt.figure(figsize = (15,9))

gs = fig.add_gridspec(3,7)
axs = [fig.add_subplot(gs[0:3, 0:3])]
im0 = axs[0].imshow(np.abs(rho),cmap=mplib.colormaps['Greys_r'],origin='lower')
axs[0].xaxis.set_ticklabels([])
axs[0].yaxis.set_ticklabels([])
for j in range(3):
    for k in range(3,7):
        axs.append(fig.add_subplot(gs[j,k]))
        axs[-1].xaxis.set_ticklabels([])
        axs[-1].yaxis.set_ticklabels([])
for j in range(nrcoils):
    axs[1+j].imshow(complex_to_rgb(coils[j,:,:]),origin='lower')
plt.show()
"""

### ... or run solver by stopping rule

In [ ]:
for reco, reco_data in solver.while_(stoprule):
    rho, coils = smoother.codomain.split(smoother(reco))
    #rho, coils = normalize(rho,coils)

    fig = plt.figure(figsize = (15,9))

    gs = fig.add_gridspec(3,7)
    axs = [fig.add_subplot(gs[0:3, 0:3])]
    axs[0].imshow(np.abs(rho),cmap=mplib.colormaps['Greys_r'],origin='lower')
    axs[0].xaxis.set_ticklabels([])
    axs[0].yaxis.set_ticklabels([])
    for j in range(3):
        for k in range(3,7):
            axs.append(fig.add_subplot(gs[j,k]))
            axs[-1].xaxis.set_ticklabels([])
            axs[-1].yaxis.set_ticklabels([])
    for j in range(nrcoils):
        axs[1+j].imshow(complex_to_rgb(coils[j,:,:]),origin='lower')
    plt.show()